# **Road Following - Collecting Data (Multi-Target)**
Use ``regression`` to predict the target location and implement line patrol

In [ ]:
#Add search path
import sys
sys.path.append('../')
# IPython Libraries for display and widgets
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from jupyter_clickable_image_widget import ClickableImageWidget
from IPython.display import display


# Python basic pakcages for image annotation
from uuid import uuid1 #命名唯一ID
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

# Camera for Jetracer
from camera import Camera, bgr8_to_jpeg

### **Open camera**

In [ ]:
image_width = 224#
image_height = 224#
display_width = 224
display_height = 224
camera_fps = 30

In [ ]:
camera = Camera(width=image_width, height=image_height, fps=camera_fps)

### **Define target collector**
Define collector class

In [ ]:
class CreateTargetWidget():
    def __init__(self, camera, display_width, display_height, name):

        self.camera = camera
        self.width = display_width
        self.height = display_height
        self.name = name

        self.target_widget = ClickableImageWidget(width=self.width, height=self.height)
        self.x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, value=0.0)
        self.y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, value=0.0)
        time.sleep(1)

        traitlets.dlink((self.camera, 'value'), (self.target_widget, 'value'), transform=self.display_xy)
        self.target_widget.on_msg(self.change_xy)



    def display_xy(self, camera_image):
        #getting data
        image = np.copy(camera_image)
        image = cv2.resize(image, (self.width, self.height))
        #support line
        image = cv2.line(image, (int(self.width/2), 0), (int(self.width/2), self.height-1), (200, 200, 200),2)
        image = cv2.line(image, (0, int(self.height/2)), (self.width-1, int(self.height/2)), (200, 200, 200),2)

        x = self.x_slider.value
        y = self.y_slider.value
        x = int((x+1) * self.width / 2)
        y = int((y+1) * self.height / 2)
        image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
        image = cv2.circle(image, (int(self.width/2), self.height-1), 8, (0, 0,255), 3)
        image = cv2.line(image, (x,y), (int(self.width/2), self.height-1), (255,0,0), 3)
        jpeg_image = bgr8_to_jpeg(image)
        return jpeg_image

    def change_xy(self, _, content, msg):
        if content['event'] == 'click':
            data = content['eventData']
            x = round(data['offsetX']/(self.width/2) - 1, 2)
            #y = round((data['offsetY'] - 112)*2/224, 2)
            y = round(data['offsetY']/(self.height/2) - 1, 2)
            #text.value += 'offset: (%d, %d) slider: (%.2f, %.2f)'%(data['offsetX'],data['offsetY'], x, y)
            #text.value += '(%d, %d)\n'%(data['offsetX'],data['offsetY'])
            self.x_slider.value = x
            self.y_slider.value = y
    def display(self):
        return ipywidgets.VBox([
            ipywidgets.Label('-'*20+self.name+'-'*20),
            self.target_widget,
            self.x_slider,
            self.y_slider
        ])

    def value(self):
        return [self.x_slider.value, self.y_slider.value]


Instantiate the target collector



In [ ]:
target_names = ['main(left)', 'secondary(right)']

target_widgets = []
for name in target_names:
    target_widgets.append(CreateTargetWidget(camera, display_width, display_height, name))

visualization

In [ ]:
display(ipywidgets.HBox([
    target_widgets[0].display(),
    target_widgets[1].display()
]))

### **save data**


In [ ]:
DATASET_DIR = 'data/dataset_xy_1'

# create folder
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

# counting tool
count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))
# save
save_widget = widgets.Button(description='SAVE',button_style='warning')

def xy_uuid(target_widgets):
    save_name = 'xy_'
    for target in target_widgets:
        data = target.value()
        save_name +='%03d_%03d_'%(data[0] * 50 + 50, data[1] * 50 + 50)
    save_name += '%s'%(uuid1())
    #return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())
    return save_name

#save images
def save_snapshot():
    uuid = xy_uuid(target_widgets)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        image = np.copy(camera.value)
        #image = correctImage(image, coefficient_group)
        #image = cv2.resize(image, (save_width, save_height))
        image = bgr8_to_jpeg(image)
        f.write(image)
    count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

def widget_save(change):
    save_snapshot()

#save function
save_widget.on_click(widget_save)

Directories not created becasue they already exist


In [ ]:
camera.stop()